In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import cv2
import tensorflow as tf

# Set some parameters
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1
TRAIN_DATA_DIR='data/train_data/x/x/'
TRAIN_MASK_DIR='data/train_data/y/y/'
TEST_DATA_DIR='data/train_data/x/x/'
TEST_MASK_DIR='data/train_data/y/y/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

Using TensorFlow backend.


In [2]:
# Get train and test IDs
train_data_ids = next(os.walk(TRAIN_DATA_DIR))[2]
train_mask_ids = next(os.walk(TRAIN_MASK_DIR))[2]

test_data_ids = next(os.walk(TEST_DATA_DIR))[2]
test_mask_ids = next(os.walk(TEST_MASK_DIR))[2]

len(train_data_ids)

31

In [9]:
# Get and resize train images and masks
X_train = np.zeros((len(train_data_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
Y_train = np.zeros((len(train_mask_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_data_ids), total=len(train_data_ids)):
    path = TRAIN_DATA_DIR + id_
    print(path)
    img = imread(path)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img

print("TER")
mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing mask images ... ')
for n, mask_id in tqdm(enumerate(train_mask_ids), total=len(train_mask_ids)):
    mask_ = imread(TRAIN_MASK_DIR + mask_id)
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH),mode='constant', preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
    Y_train[n] = mask

Getting and resizing train images ... 


  0%|          | 0/31 [00:00<?, ?it/s]

data/train_data/x/x/10.tif


  3%|▎         | 1/31 [00:00<00:03,  7.65it/s]

data/train_data/x/x/11.tif
data/train_data/x/x/12.tif
data/train_data/x/x/13.tif
data/train_data/x/x/14.tif
data/train_data/x/x/15.tif
data/train_data/x/x/16.tif


...

 42%|████▏     | 13/31 [00:00<00:01, 13.47it/s]

data/train_data/x/x/23.tif
data/train_data/x/x/24.tif
data/train_data/x/x/25.tif
data/train_data/x/x/26.tif
data/train_data/x/x/27.tif
data/train_data/x/x/28.tif


 61%|██████▏   | 19/31 [00:00<00:00, 17.37it/s]

data/train_data/x/x/29.tif
data/train_data/x/x/3.tif
data/train_data/x/x/30.tif
data/train_data/x/x/31.tif
data/train_data/x/x/32.tif
data/train_data/x/x/33.tif


ValueError: could not broadcast input array from shape (512,512,512) into shape (512,512)

In [ ]:
# Get and resize train images and masks
X_train = np.zeros((len(train_data_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_mask_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_data_ids), total=len(train_data_ids)):
    path = TRAIN_DATA_DIR + id_
    img = imread(path)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_CUBIC)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    
    for n, mask_id in tqdm(enumerate(train_mask_ids), total=len(train_mask_ids)):
        mask_ = imread(TRAIN_MASK_DIR + mask_id)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_CUBIC), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [6]:
enumerate(train_data_ids)